## Imports

In [25]:
import pickle
import pandas as pd

from backtesting import Backtest
from backtesting import Strategy

### Call the model

In [2]:
with open('models/model_dt_classification.pkl', 'rb') as f:
    model_dt = pickle.load(f)

In [3]:
model_dt

DecisionTreeClassifier(max_depth=15)

### Get the data

In [5]:
df = pd.read_excel('data/MSFT_linkedin_processed.xlsx', parse_dates=['Date'], index_col=0)
df.head()

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2016-12-08,61.299999,61.580002,60.840000,61.009998,21220800,1.549141,UP
2016-12-09,61.180000,61.990002,61.130001,61.970001,27349400,0.321694,UP
2016-12-12,61.820000,62.299999,61.720001,62.169998,20198100,1.286125,UP
2016-12-13,62.500000,63.419998,62.240002,62.980000,35718900,-0.478620,DOWN
2016-12-14,63.000000,63.450001,62.529999,62.680000,30352700,-0.159793,DOWN


### Create Strategy Class

In [9]:
df_explanatory = df.drop(columns=['change_tomorrow','change_tomorrow_direction'])

In [14]:
explanatory_today = df_explanatory.iloc[[-1],:]
explanatory_today

,Open,High,Low,Close,Volume
Date,,,,,
2023-12-14,373.309998,373.76001,364.130005,365.929993,43277500


In [15]:
forecast_tomorrow = model_dt.predict(explanatory_today)[0]
forecast_tomorrow

'UP'

Our trading strategy is easy. If we predict that stock price will go up we buy and if we predict that stock price will go down then we sell

In [16]:
list_buy_sell = []
already_bought = False

for direction_tomorrow in df.change_tomorrow_direction:
    if direction_tomorrow == 'UP' and already_bought == False:
        list_buy_sell.append(1)
        already_bought = True
    elif direction_tomorrow == 'DOWN' and already_bought == True:
        list_buy_sell.append(-1)
        already_bought = False
    else:
        list_buy_sell.append(0)

df['buy_sell'] = list_buy_sell

df

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction,buy_sell
Date,,,,,,,,
2016-12-08,61.299999,61.580002,60.840000,61.009998,21220800,1.549141,UP,1
2016-12-09,61.180000,61.990002,61.130001,61.970001,27349400,0.321694,UP,0
2016-12-12,61.820000,62.299999,61.720001,62.169998,20198100,1.286125,UP,0
2016-12-13,62.500000,63.419998,62.240002,62.980000,35718900,-0.478620,DOWN,-1
2016-12-14,63.000000,63.450001,62.529999,62.680000,30352700,-0.159793,DOWN,0
...,...,...,...,...,...,...,...,...
2023-12-08,369.200012,374.459991,368.230011,374.230011,20144800,-0.789126,DOWN,-1
2023-12-11,368.480011,371.600006,366.100006,371.299988,27708800,0.822698,UP,1
2023-12-12,370.850006,374.420013,370.459991,374.380005,24838300,-0.002674,DOWN,-1


In [26]:
class SimpleClassificationUP(Strategy):
    
    def init(self):
        self.model = model_dt
        self.already_bought = False
    
    def next(self):
        explanatory_today = self.data.df.iloc[[-1],:]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]

        if forecast_tomorrow == 'UP' and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow == 'DOWN' and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

Define initial conditions

In [27]:
bt = Backtest(data= df_explanatory, strategy=SimpleClassificationUP, cash=10000,
              commission=.002, exclusive_orders=True)

Run backtesting

In [28]:
results = bt.run()

results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2023-12-14 00:00:00
Duration,2562 days 00:00:00
Exposure Time [%],99.88675
Equity Final [$],105003706.547121
Equity Peak [$],110686156.049352
Return [%],1049937.065471


Return is so high because the model already knew what happend to MSFT stock. 